<center>
  
# TABSYN: Tabular Data Synthesis with Diffusion Models

</center>

Two challenges regarding the extention of diffusion models to tabular data are:
1. **Diverse data types:** a single table can have different columns each containing data of different types, including numerical, categorical, text, etc.
2. **Varied distributions:** the distribution of data under different columns in a single table varry widely from column to column.

**TabSyn** addresses these challenges by introducing a latent space where tabular data of all columns are jointly represented. It then proceedes to train a diffusion model on the latent representations.
This tactic allows TabSyn to:
1. Train a single diffusion model for all data types in the dataset (i.e. Generality).
2. Optimize the distribution of latent embeddings to facilitate training of the subsequent diffusion model, thus generating higher quality synthetic data (i.e. Quality).
3. Require much fewer reverse steps during training of the diffusion model, and synthesize data faster (i.e. Speed).

In this notebook, we review and implement the TabSyn model. The notebook is organized as follows:

1. [Imports and Setup]()


2. [Default Dataset]()
    
    
3. [TabSyn Algorithm]()
    
    3.1. [Load Config]()
    
    3.2. [Make Dataset]()
    
    3.3. [Instantiate Model]()
    
    3.4. [Train Model]()
        
    3.5. [Load Pretrained Model]()
    
    3.6. [Sample Data]()
    
    3.7. [Review Synthetic Data]()


# Imports and Setup

In this section, we import all necessary libraries and modules required for setting up the environment.
Most of the libraries we need to implement TabSyn are the same as TabDDPM.
We also specify `NAME_URL_DICT_UCI`, `DATA_NAME`, `DATA_DIR` and other paths as in TabDDPM's implementation.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.getcwd()

'/fs01/home/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis'

In [ ]:
# os.chdir('/fs01/home/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis/')

In [4]:
import os
from scripts.process_dataset import process_data

In [5]:
import pandas as pd

df = pd.read_csv('/h/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis/data/raw_data/IBM_AML_FeatureEngineered_FS1/flattened_sample.csv')
# Ignore the The Patern ID and The Source columns
df=df.iloc[::,1::]   
# Convert the first 29 Destination and Currency columns to boolean
df.iloc[:, :29] = df.iloc[:, :29].astype(bool)

# Convert the last column Is Fanout to boolean
df.iloc[:, -1] = df.iloc[:, -1].astype(bool)

/tmp/ipykernel_21941/4202322190.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, :29] = df.iloc[:, :29].astype(bool)


In [6]:
df.head()

,Source,Destination_1,Destination_2,Destination_3,Destination_4,Destination_5,Destination_6,Destination_7,Destination_8,Destination_9,...,Yen,UK Pound,number_transactions_above_9k_cad,avg_transaction_value_in_cad,transaction_value_variance_in_cad,variance_from_10k_cad,min_day_to_max_day_range,avg_transaction_frequency_in_days,transaction_frequency_variance_in_days,Is FanOut
0,True,True,True,False,False,False,False,False,False,False,...,False,0,0,5873.10725,5.063292e+05,3.456882e+07,6,3.000000,1.66884,False
1,True,True,True,True,True,True,True,True,True,True,...,False,0,13,648570.30320,7.507306e+11,1.894381e+13,5,0.294118,1.66884,False
2,True,True,True,True,True,True,True,True,True,True,...,False,0,9,649125.71180,8.189731e+11,1.391048e+13,1,0.083333,1.66884,False
3,True,True,True,False,False,False,False,False,False,False,...,False,0,0,472.97624,4.133341e+05,1.819417e+08,0,0.000000,1.66884,False
4,True,True,False,False,False,False,False,False,False,False,...,False,0,0,15.66720,NaN,9.968690e+07,0,0.000000,1.66884,False


In [7]:
def get_column_info(df):
    column_info = {col: str(df[col].dtype).replace('float64', 'float').replace('int64', 'int').replace('object', 'str') for col in df.columns}
    return column_info

# Get column info
column_info = get_column_info(df)

# Print in desired format
print({"column_info": column_info})

{'column_info': {'Source': 'bool', 'Destination_1': 'bool', 'Destination_2': 'bool', 'Destination_3': 'bool', 'Destination_4': 'bool', 'Destination_5': 'bool', 'Destination_6': 'bool', 'Destination_7': 'bool', 'Destination_8': 'bool', 'Destination_9': 'bool', 'Destination_10': 'bool', 'Destination_11': 'bool', 'Destination_12': 'bool', 'Destination_13': 'bool', 'Destination_14': 'bool', 'Destination_15': 'bool', 'Destination_16': 'bool', 'US Dollar': 'bool', 'Euro': 'bool', 'Yuan': 'bool', 'Rupee': 'bool', 'Ruble': 'bool', 'Canadian Dollar': 'bool', 'Australian Dollar': 'bool', 'Brazil Real': 'bool', 'Swiss Franc': 'bool', 'Shekel': 'bool', 'Bitcoin': 'bool', 'Yen': 'bool', 'UK Pound': 'int', 'number_transactions_above_9k_cad': 'int', 'avg_transaction_value_in_cad': 'float', 'transaction_value_variance_in_cad': 'float', 'variance_from_10k_cad': 'float', 'min_day_to_max_day_range': 'int', 'avg_transaction_frequency_in_days': 'float', 'transaction_frequency_variance_in_days': 'float', 'I

## Prepare the Dataset Config File

In [ ]:
df.columns

In [8]:
import pandas as pd
import json

def generate_json_config(df):
    # Get column names
    column_names = df.columns.tolist()
    
    # Identify the target column index (last column)
    target_col_idx = [len(column_names) - 1]
    
    # Initialize lists for numerical and categorical column indices
    num_col_idx = []
    cat_col_idx = []
    
    # Initialize dictionary for column info
    column_info = {}
    
    # Determine column types and indices
    for idx, col in enumerate(df.columns):
        col_type = df[col].dtype
        
        # Fill in column_info with appropriate type
        if pd.api.types.is_integer_dtype(col_type):
            column_info[col] = "int"
            if idx != target_col_idx[0]:  # Exclude target column from num_col_idx
                num_col_idx.append(idx)
        elif pd.api.types.is_float_dtype(col_type):
            column_info[col] = "float"
            num_col_idx.append(idx)
        elif pd.api.types.is_bool_dtype(col_type):
            column_info[col] = "bool"
            cat_col_idx.append(idx)
        else:
            column_info[col] = "str"
            cat_col_idx.append(idx)
    
    # Calculate train_num and test_num
    total_rows = len(df)
    train_num = int(total_rows * 0.8)
    test_num = total_rows - train_num
    
    # Construct JSON configuration object
    config = {
        "column_names": column_names,
        "num_col_idx": num_col_idx,
        "cat_col_idx": cat_col_idx,
        "target_col_idx": target_col_idx,
        "file_type": "csv",
        "data_path": "/h/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis/data/processed_data/flattened_sample.csv", 
        "column_info": column_info,
        "train_num": train_num,
        "test_num": test_num
    }
    
    # Print JSON object
    print(json.dumps(config, indent=4))

In [14]:
generate_json_config(df)

{
    "column_names": [
        "Source",
        "Destination_1",
        "Destination_2",
        "Destination_3",
        "Destination_4",
        "Destination_5",
        "Destination_6",
        "Destination_7",
        "Destination_8",
        "Destination_9",
        "Destination_10",
        "Destination_11",
        "Destination_12",
        "Destination_13",
        "Destination_14",
        "Destination_15",
        "Destination_16",
        "US Dollar",
        "Euro",
        "Yuan",
        "Rupee",
        "Ruble",
        "Canadian Dollar",
        "Australian Dollar",
        "Brazil Real",
        "Swiss Franc",
        "Shekel",
        "Bitcoin",
        "Yen",
        "UK Pound",
        "number_transactions_above_9k_cad",
        "avg_transaction_value_in_cad",
        "transaction_value_variance_in_cad",
        "variance_from_10k_cad",
        "min_day_to_max_day_range",
        "avg_transaction_frequency_in_days",
        "transaction_frequency_variance_in_days

In [15]:
df.shape

(131, 38)

# AML Dataset

For more explanation of different steps in this section, please refer to TabDDPM's notebook.

In [18]:
DATA_DIR = "data/"
RAW_DATA_DIR = os.path.join(DATA_DIR, "raw_data")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed_data")
SYNTH_DATA_DIR = os.path.join(DATA_DIR, "synthetic_data")
DATA_NAME = "adult"

MODEL_PATH = "models/tabsyn"



In [19]:
# process data
INFO_DIR = "data_info"
process_data(DATA_NAME, INFO_DIR, DATA_DIR)


XXXXXXXXXXXXXXXXXX(32561, 15)


FileNotFoundError: [Errno 2] No such file or directory: 'data/raw_data/adult/test.data'

In [ ]:
INFO_DIR = "data_info"
# DONT NEED process_data(DATA_NAME, INFO_DIR, DATA_DIR)

# review data
df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, DATA_NAME, "train.csv"))

In [ ]:
# review json file and its contents
with open(f"{PROCESSED_DATA_DIR}/{DATA_NAME}/info.json", "r") as file:
    data_info = json.load(file)
# pprint(data_info)

# TabSyn Algorithm

In this section, we will describe the design of TabSyn as well as its main hyperparameters loaded through config, which affect the model’s effectiveness. 

**TabSyn** consists of two parts:
1. A *variational auto-encoder (VAE)* which learns a joint representation space for the given tabular data.
2. A *Diffusion model* which learns the distribution of data in the joint representation space.

The figure below shows a diagram of the TabSyn model.

<p align="center">
<img src="figures/tabsyn.jpg" width="1000"/>
</p>

**VAE**

The left-side of the figure shows the VAE which operates in the original data space. The VAE itself consists of two parts: an encoder and a decoder. It also contains the corresponding tokenizer and detokenizer.
Each row of the input tabular data ($\pmb{x}$) is tokenized, then embedded by a transformer. Another transformer decodes the embeddings and a detokenizer reconstructs the table ($\pmb{\tilde{x}}$). The VAE is trained by minimizing the reconstruction loss between $\pmb{x}$ and $\pmb{\tilde{x}}$.

After the VAE is fully trained, the whole data ($\pmb{x}$) is tokenized and embedded. The embedding of each row is flattened to form a 1-dimensional vector $\pmb{z}$.
These 1-dimensional embeddings for all rows are stored on disk, and will later be used to train the diffusion model.

**Diffusion**

The right-side of the figure shows the diffusion model which operates in the latent representation space; in other words, it only *sees* the embeddings obtained by the VAE, not the original tabular data.
The diffusion model can be similarly divided into two parts: a forward process, and a reverse process.

The forward process receives the embedded data points. A single data point is denoted by $\pmb{z_0}$ in the figure. Gaussian noise is incrementally added to the embeddings in numerous incremental steps during the forward process. The number of the steps is denoted by $T$ in the figure. $T$ should be high enough that the distribution of embeddings at step $t=T$ is essentially a standard Gaussian distribution; in other words, the signal-to-noise ratio is practically zero.

The reverse process, on the other hand, learns to *predict* an earlier-step embedding (e.g. $\pmb{z_{t-\Delta t}}$) from a later-step embedding (e.g. $\pmb{z_t}$) via a neural network.

After the diffusion model is fully trained, the reverse process can estimate the data distribution at step $t=0$ if it receives a standard Gaussian distribution at step $t=T$. New data points can be synthesized by sampling from this estimated distribution.


## Load Config

In this section, we will load the configuration file that contains the hyperparameters for the TabSyn model. 

In [ ]:
config_path = os.path.join("src/baselines/tabsyn/configs", f"{DATA_NAME}.toml")
raw_config = src.load_config(config_path)

pprint(raw_config)

The configuration file is a TOML file that contains the following hyperparameters:

1. **model_params:** specifies the structure of the transformers (both encoder and decoder) in the VAE model, including number of transformer layers, number of self-attnetion heads and token dimension.

2. **transforms:** specifies the transformations and preprocessing of the data before tokenization, such as cleaning, normalization, and encoding.
    - For preprocessing numerical features, we use the gaussian quantile transformation and replace the NaN values with mean of each row.
    - For categorical features, we use the one-hot encoding method. NaN values are left unchanged, but we have the option to replace them. We have the option to drop the values that appear with less than a given minimum frequency under each column. Furthermore, we have the option to add an extra encoding step for categorical features during tokenization.

3. **train.vae:** specifies training parameters of the VAE, including batch size, number of epochs, and number of dataset workers.

4. **train.diffusion:** specifies the same training parameters as above for the diffusion model.

5. **train.optim.vae:** specifies the parameters of the *Adam* optimizer and the `ReduceLROnPlateau` learning rate scheduler used to train the VAE. Optimizer parameters include initial learning rate and weight decay. LR scheduler parameters includer `factor` and `patience`.

6. **train.optim.diffusion:** specifies the same parameters as above for the diffusion model.

7. **loss_params:** specifies parameters of the loss function used to train the VAE including `max_beta`, `min_beta` and `lambd`.

$\beta$ is the coefficient of the KL divergence term in the VAE loss formula,

$\mathcal{L}_{vae} = \mathcal{L}_{mse} + \mathcal{L}_{ce} + \beta \mathcal{L}_{kl}$
.

Parameters `max_beta` and `min_beta` determine the range of $\beta$. $\beta$ is first set to `max_beta`. If the loss stops decreasing for a certain number of epochs (e.g. $10$ epochs), then at the end of each epoch after that (e.g. epoch $11$, $12$, etc.) $\beta$ is decreased by a factor of `lambd`,
$\beta_{new} = \lambda \beta_{curr}$,
until it reaches `beta_min`.


## Make Dataset

In this section, we pre-process the data and make a dataset object.

First, we determine transformations needed for the dataset, such as normalization and cleaning, in `transforms`. Next, using `preprocess` function we load the data from disk in arrays that contain both training and test data (`X_num` and `X_cat`), as well as the number of categories for each categorical feature (`categories`) and the number of numerical features (`d_numerical`).

We then separate the train and test data in different arrays and convert them to Pytorch tensors.
We create a dataset object (`TabularDataset`) with the train data. `TabularDataset` is a simple module which returns the tokens of a single row at a time. Each row constiutes a single data sample in TabSyn. Afterwards, we create a Dataloader for the train data using the `batch_size` and `num_workers` specified in config.

In contrast, we keep the test data as tensors (`X_test_num` and `X_test_cat`). If a GPU is available, we move these tensors to GPU so that they can be accessed by the model later on.

In [ ]:
# preprocess data
X_num, X_cat, categories, d_numerical = preprocess(os.path.join(PROCESSED_DATA_DIR, DATA_NAME),
                                                   transforms = raw_config["transforms"],
                                                   task_type = raw_config["task_type"])

# separate train and test data
X_train_num, X_test_num = X_num
X_train_cat, X_test_cat = X_cat

# convert to float tensor
X_train_num, X_test_num = torch.tensor(X_train_num).float(), torch.tensor(X_test_num).float()
X_train_cat, X_test_cat =  torch.tensor(X_train_cat), torch.tensor(X_test_cat)

# create dataset module
train_data = TabularDataset(X_train_num.float(), X_train_cat)

# move test data to gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_test_num = X_test_num.float().to(device)
X_test_cat = X_test_cat.to(device)

# create train dataloader
train_loader = DataLoader(
    train_data,
    batch_size = raw_config["train"]["vae"]["batch_size"],
    shuffle = True,
    num_workers = raw_config["train"]["vae"]["num_dataset_workers"],
)

## Instantiate Model

Next, we instantiate the model using the `TabSyn` class. `TabSyn` class takes the following arguments:

1. `train_loader`: dataloader for train data.
2. `X_test_num`: numerical features of the test data.
3. `X_test_cat`: categorical features of the train data.
4. `num_numerical_features`: number of numerical features in the dataset.
5. `num_classes`: number of classes (i.e. categories) of each categorical feature in the dataset.
6. `device`: the device on which the model and data exist, either "cpu" or "cuda".

In [ ]:
tabsyn = TabSyn(train_loader,
                X_test_num, X_test_cat,
                num_numerical_features = d_numerical,
                num_classes = categories,
                device = device)

`TabSyn` class has the tools to instantiate VAE and diffusion models, train both, and sample from the trained diffusion model.
We will demonstrate how to use these tools in the following sections.

## Train Model


The VAE and the diffusion model are trained independently. The following subsections explain each training process.


### A. Train VAE

First, we need to instantiate the VAE using the `instantiate_vae` method. This method takes the VAE model hyperparameters, optimizer and lr scheduler parameters from config, and instantiates them.

In [ ]:
# instantiate VAE model for training
tabsyn.instantiate_vae(**raw_config["model_params"], optim_params = raw_config["train"]["optim"]["vae"])

Now that we have instantiated the VAE, we can train it using the `train_vae` function.
This function receives the loss hyperparameters and number of epochs from the config.
Moreover, it recieves `save_path` which is the directory where trained model checkpoints will be saved.

In [ ]:
os.makedirs(f"{MODEL_PATH}/{DATA_NAME}/vae")
tabsyn.train_vae(**raw_config["loss_params"],
                 num_epochs = raw_config["train"]["vae"]["num_epochs"],
                 save_path = os.path.join(MODEL_PATH, DATA_NAME, "vae"))

After training the VAE, we embed the training data with the trained encoder and store the embeddings in a direcotry specified by `vae_ckpt_dir`.

In [ ]:
# embed all inputs in the latent space
tabsyn.save_vae_embeddings(X_train_num, X_train_cat,
                           vae_ckpt_dir = os.path.join(MODEL_PATH, DATA_NAME, "vae"))

### B. Train Diffusion Model

Now that we have stored the training data embeddings, we need to load and prepare them for the diffusion model.
We load the embeddings using `load_vae_embeddings`. We normalize the embeddings by subtracting the mean and dividing by the standard deviation. Then, we create a Dataloader with the specified `batch_size` and `num_workers` from the config.

In [ ]:
# load latent space embeddings
train_z, _ = tabsyn.load_latent_embeddings(os.path.join(MODEL_PATH, DATA_NAME, "vae"))  # train_z dim: B x in_dim

# normalize embeddings
mean, std = train_z.mean(0), train_z.std(0)
train_z = (train_z - mean) / std
latent_train_data = train_z

# create data loader
latent_train_loader = DataLoader(
    latent_train_data,
    batch_size = raw_config["train"]["diffusion"]["batch_size"],
    shuffle = True,
    num_workers = raw_config["train"]["diffusion"]["num_dataset_workers"],
)

Now that the data is ready, we instantiate the diffusion model with `instantiate_diffusion`. The input dimension and hidden dimention of the diffusion model is determined by the dimension of the embeddings. 
Moreover, we instantiate the optimizer and lr scheduler using hyperparameters from config.

In [ ]:
# instantiate diffusion model for training
tabsyn.instantiate_diffusion(in_dim = train_z.shape[1], hid_dim = train_z.shape[1], optim_params = raw_config["train"]["optim"]["diffusion"])

We train the diffusion model with `train_diffusion` function.
This function takes the following arguements:
1. `latent_train_loader`: dataloader for the latent representations which are used to train the diffusion model.
2. `num_epochs`: number of training epochs.
3. `ckpt_path`: directory where the model checkpoints will be stored.

In [ ]:
os.makedirs(f"{MODEL_PATH}/{DATA_NAME}")
# train diffusion model
tabsyn.train_diffusion(latent_train_loader,
                       num_epochs = raw_config["train"]["diffusion"]["num_epochs"],
                       ckpt_path = os.path.join(MODEL_PATH, DATA_NAME))

## Load Pretrained Model

Instead of training model from scratch, we can also load weights of a pre-trained model from a given checkpoint with `load_model_state` function.
If we haven't instantiated the VAE and diffusion model beforehand, we need to instantiate them first using `instantiate_vae` and `instantiate_diffusion` methods.

In [ ]:
# instantiate VAE model
tabsyn.instantiate_vae(**raw_config["model_params"], optim_params = None)

latent_embeddings_path = "/projects/diffusion_bootcamp/models/tabular/tabsyn/default/vae"
# load latent embeddings of input data
train_z, token_dim = tabsyn.load_latent_embeddings(latent_embeddings_path)

# instantiate diffusion model
tabsyn.instantiate_diffusion(in_dim = train_z.shape[1], hid_dim = train_z.shape[1], optim_params = None)

pretrained_model_path = "/projects/diffusion_bootcamp/models/tabular/tabsyn/default"
# load state from checkpoint
tabsyn.load_model_state(ckpt_dir = pretrained_model_path,
                        dif_ckpt_name = "model.pt")

## Sample Data

Now that we trained the model effectively, using `sample` function we can generate synthetic data starting from compelete noise. The input of this function is as follows:

1. `train_z`: latent embeddings of the training data.
2. `info`: info about the data from the json file we reviewed at the beginning of this notebook.
3. `num_inverse`: detokenizer for numerical features.
4. `cat_inverse`: detokenizer for categorical features.
5. `save_path`: file-path where the synthetic table will be saved.

In [ ]:
# load data info file
with open(os.path.join(PROCESSED_DATA_DIR, DATA_NAME, "info.json"), "r") as file:
    data_info = json.load(file)
data_info["token_dim"] = token_dim

# get inverse tokenizers
_, _, categories, d_numerical, num_inverse, cat_inverse = preprocess(os.path.join(PROCESSED_DATA_DIR, DATA_NAME),
                                                                     transforms = raw_config["transforms"],
                                                                     task_type = raw_config["task_type"],
                                                                     inverse = True)

# sample data
num_samples = train_z.shape[0]
in_dim = train_z.shape[1] 
mean_input_emb = train_z.mean(0)
tabsyn.sample(num_samples,
              in_dim,
              mean_input_emb,
              info = data_info,
              num_inverse = num_inverse,
              cat_inverse = cat_inverse,
              save_path = os.path.join(SYNTH_DATA_DIR, DATA_NAME, "tabsyn.csv"))

## Review Synthetic Data

Finally here, we review the synthesized data. In the following `evaluate_synthetic_data.ipynb` notebook, we will evaluate this synthesized data with respect to various metrics.

In [ ]:
df = pd.read_csv(os.path.join(SYNTH_DATA_DIR, DATA_NAME, "tabsyn.csv"))

# Display the first few rows of the DataFrame
visualize_default(df).head(10)

## References

**Zhang, Hengrui, et al.** "Mixed-type tabular data synthesis with score-based diffusion in latent space." *International Conference on Learning Representations (ICLR)* (2023).

**GitHub Repository:** [Amazon Science - Tabsyn](https://github.com/amazon-science/tabsyn)